In [1]:
# 1：ライブラリのインポート--------------------------------
import numpy as np #numpyという行列などを扱うライブラリを利用
import pandas as pd #pandasというデータ分析ライブラリを利用
import matplotlib.pyplot as plt #プロット用のライブラリを利用
from sklearn import cluster, preprocessing #機械学習用のライブラリを利用
from sklearn import datasets #使用するデータ

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
def compute_bic(kmeans,X):
    """
    Computes the BIC metric for a given clusters

    Parameters:
    -----------------------------------------
    kmeans:  List of clustering object from scikit learn

    X     :  multidimension np array of data points

    Returns:
    -----------------------------------------
    BIC value
    """
    # assign centers and labels
    centers = [kmeans.cluster_centers_]
    labels  = kmeans.labels_
    #number of clusters
    m = kmeans.n_clusters
    # size of the clusters
    n = np.bincount(labels)
    #size of data set
    N, d = X.shape

    #compute variance for all clusters beforehand
    cl_var = (1.0 / (N - m) / d) * sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 'euclidean')**2) for i in range(m)])

    const_term = 0.5 * m * np.log(N) * (d+1)

    BIC = np.sum([n[i] * np.log(n[i]) -
               n[i] * np.log(N) -
             ((n[i] * d) / 2) * np.log(2*np.pi*cl_var) -
             ((n[i] - 1) * d/ 2) for i in range(m)]) - const_term
    
    BIC = BIC*(-1)

    return(BIC, labels, m)

In [4]:
df = pd.read_csv(r"../data/src/ch4/athome_bunkyo.csv",usecols = ["物件番号","公開月","徒歩(分)","所在階","Area","賃料・価格","how_old"],dtype="str")
df = df[df["公開月"].str.contains("2015")].dropna()
df = df.drop_duplicates(subset = ["物件番号"])
del df["物件番号"], df["公開月"]
df["所在階"] = df["所在階"].replace('B1', '-1')
df = df.astype(float)
df.head(1)

,徒歩(分),賃料・価格,所在階,Area,how_old
40,9.0,65000.0,3.0,20.0,374.4


In [5]:
test_array = np.array([df["徒歩(分)"].tolist(),df["所在階"].tolist(),df["Area"].tolist(),df["how_old"].tolist(),df["賃料・価格"].tolist()], np.float).T

#標準化
sc=preprocessing.StandardScaler()
sc.fit(test_array)
X_norm=sc.transform(test_array)
X_norm

array([[ 0.97028821, -0.44073542, -0.35844986,  1.02610327, -0.78448471],
       [ 1.27347573, -0.76640128, -0.12263059, -0.74426461, -0.32741153],
       [-1.15202442,  0.86192804, -0.39464275,  0.81866668, -0.72735056],
       ...,
       [ 0.36391317, -1.09206714,  0.34137352, -1.35998547,  0.45342182],
       [-0.24246187, -1.09206714,  0.4078214 ,  0.437874  ,  0.26297467],
       [-0.24246187, -1.09206714,  0.10725079,  1.8070009 , -0.11791965]])

In [6]:
Spec_CL = [cluster.SpectralClustering(n_clusters=i, affinity="nearest_neighbors").fit(X_norm) for i in range(2,8)]
print("Spec_CL done")

C:\Users\mymt_akck\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Spec_CL done


In [7]:
for scl in Spec_CL:
    print(pd.DataFrame(scl.labels_)[0].value_counts())

0    34310
1       85
Name: 0, dtype: int64
0    34329
2       40
1       26
Name: 0, dtype: int64
0    34326
1       38
3       20
2       11
Name: 0, dtype: int64
0    21061
2    13090
1      191
3       30
4       23
Name: 0, dtype: int64
0    22099
2     6996
5     5022
3      191
4       50
1       37
Name: 0, dtype: int64
0    34090
1      191
5       42
6       28
2       20
4       14
3       10
Name: 0, dtype: int64


In [8]:
#ラベルを付けてOLS実行
df["Spcl_number"] = Spec_CL[4].labels_

In [9]:
df.head(2)

,徒歩(分),賃料・価格,所在階,Area,how_old,Spcl_number
40,9.0,65000.0,3.0,20.00,374.400000,2
41,10.0,89000.0,2.0,28.34,114.666667,2


In [11]:
#全データ
import statsmodels.api as sm

X = df[["徒歩(分)","所在階","how_old"]]
X = sm.add_constant(X)
Y = df["賃料・価格"]/df["Area"]
model = sm.OLS(Y, X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     2684.
Date:                Mon, 31 Dec 2018   Prob (F-statistic):               0.00
Time:                        16:00:57   Log-Likelihood:            -2.6979e+05
No. Observations:               34395   AIC:                         5.396e+05
Df Residuals:                   34391   BIC:                         5.396e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3708.6805     10.663    347.809      0.0

In [13]:
for cl_num6 in [0,1,2,3,4,5]:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("cl_n6_cl_{}".format(cl_num6))
    df_olsee = df[df["Spcl_number"] == cl_num6]
    X = df_olsee[["徒歩(分)","所在階","how_old"]]
    X = sm.add_constant(X)
    Y = df_olsee["賃料・価格"]/df_olsee["Area"]
    model = sm.OLS(Y, X)
    result = model.fit()
    print(result.summary())

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
cl_n6_cl_0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     2272.
Date:                Mon, 31 Dec 2018   Prob (F-statistic):               0.00
Time:                        16:01:50   Log-Likelihood:            -1.7393e+05
No. Observations:               22099   AIC:                         3.479e+05
Df Residuals:                   22095   BIC:                         3.479e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     281.7
Date:                Mon, 31 Dec 2018   Prob (F-statistic):          5.07e-169
Time:                        16:01:51   Log-Likelihood:                -38198.
No. Observations:                5022   AIC:                         7.640e+04
Df Residuals:                    5018   BIC:                         7.643e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3594.7681     31.588    113.800      0.0

In [14]:
scldes = df.groupby("Spcl_number")[["徒歩(分)","所在階","Area","賃料・価格","how_old"]].describe()
scldes.to_csv(r"../data/dev/ch4/athome_bunkyo_rent_scldes.csv")
scldes

徒歩(分)                                                      所在階  \
               count      mean       std  min  25%  50%    75%   max    count   
Spcl_number                                                                     
0            22099.0  5.350016  2.984253  1.0  3.0  5.0   7.00  40.0  22099.0   
1               37.0  6.648649  2.030547  4.0  6.0  6.0   9.00  10.0     37.0   
2             6996.0  8.966695  2.999910  5.0  7.0  8.0  10.00  31.0   6996.0   
3              191.0  1.000000  0.000000  1.0  1.0  1.0   1.00   1.0    191.0   
4               50.0  5.800000  1.714286  4.0  5.0  5.0   5.75   9.0     50.0   
5             5022.0  3.543011  1.374810  1.0  3.0  4.0   5.00   6.0   5022.0   

                                                              Area             \
                 mean       std  min  25%  50%  75%   max    count       mean   
Spcl_number                                                                     
0            5.617494  3.107013 -1.0  3.0  5.0  7.0  24.0  22099.0  37.300749   
1            5.081081  2.861414  3.0  3.0  4.0  4.0  10.0     37.0  36.169189   
2            2.158662  1.025053 -1.0  1.0  2.0  3.0  12.0   6996.0  23.985596   
3            3.000000  0.000000  3.0  3.0  3.0  3.0   3.0    191.0  23.019686   
4            6.600000  3.129843  1.0  7.0  7.0  9.0   9.0     50.0  33.556400   
5            1.871565  0.675069  1.0  1.0  2.0  2.0   4.0   5022.0  24.770763   

                                                                  賃料・価格  \
                   std    min     25%    50%      75%      max    count   
Spcl_number                                                               
0            43.082990   2.00  22.035  32.24  46.0000  4812.00  22099.0   
1            23.295812  20.72  21.080  25.40  25.5900    76.55     37.0   
2             7.294981   3.01  20.000  22.12  26.8525    55.50   6996.0   
3             2.889084  16.20  21.000  22.67  25.0900    29.50    191.0   
4            17.200279  19.87  25.050  25.46  30.1000    70.18     50.0   
5             7.220087   9.50  20.010  23.09  28.2400    61.12   5022.0   

                                                                      \
                      mean           std      min      25%       50%   
Spcl_number                                                            
0            119796.386579  59528.827090  20000.0  79000.0  105000.0   
1            126023.243243  66688.810075  79400.0  80100.0   96500.0   
2             80394.263865  19564.414848  27000.0  68000.0   77000.0   
3             82337.696335   9243.907976  60000.0  78000.0   82000.0   
4            120980.000000  44238.449432  79000.0  94000.0   98000.0   
5             82876.825966  19767.541349  29000.0  70000.0   80000.0   

                                  how_old                                     \
                  75%        max    count        mean         std        min   
Spcl_number                                                                    
0            148000.0  1800000.0  22099.0  233.844817  157.675292  -9.166667   
1             99500.0   245000.0     37.0   66.329730   54.562601  -2.033333   
2             88000.0   180000.0   6996.0  204.803878  123.579659  -3.066667   
3             90000.0    99000.0    191.0  151.016056   51.288051  82.200000   
4            119000.0   210000.0     50.0   78.002667   58.911336  -3.066667   
5             90000.0   171000.0   5022.0  211.844305  122.111640  -4.066667   

                                                              
                    25%         50%         75%          max  
Spcl_number                                                   
0            113.566667  184.666667  348.033333  1401.133333  
1              1.033333   83.166667   90.300000   137.966667  
2            108.533333  168.400000  305.366667   656.400000  
3            111.100000  145.066667  197.316667   285.133333  
4              0.958333   98.433333  117.125000   151.200000  
5    